In [0]:
%pip install mlflow

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.3/101.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.6 MB/s eta 0:00:00
     ━━━

In [0]:
dbutils.library.restartPython()

In [0]:
import mlflow
import mlflow.sklearn 
import pandas as pd 
import matplotlib.pyplot as plt 
from numpy import savetxt 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [0]:
%pip list

Package                  Version
------------------------ -------------
alembic                  1.13.1
aniso8601                9.0.1
appdirs                  1.4.4
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
asttokens                2.2.1
attrs                    21.4.0
backcall                 0.2.0
beautifulsoup4           4.11.1
black                    22.6.0
bleach                   4.1.0
blinker                  1.7.0
boto3                    1.24.28
botocore                 1.27.28
certifi                  2022.9.14
cffi                     1.15.1
chardet                  4.0.0
charset-normalizer       2.0.4
click                    8.1.7
cloudpickle              3.0.0
cryptography             37.0.1
cycler                   0.11.0
Cython                   0.29.32
databricks-sdk           0.1.6
dbus-python              1.2.18
debugpy                  1.6.0
decorator                5.1.1
defusedxml               0.7.1
distlib                  0.3.7
distro   

In [0]:
df = datasets.fetch_california_housing()

In [0]:
X = df.data
y = df.target

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
print(X)
print(X.shape)
print(y)
print(y.shape)

[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]]
(20640, 8)
[4.526 3.585 3.521 ... 0.923 0.847 0.894]
(20640,)


In [0]:
mlflow.sklearn.autolog()   ## to enable logging

In [0]:
with mlflow.start_run(nested=True):
    dt = DecisionTreeRegressor()
    params = {'criterion': ['squared_error', 'absolute_error'],
                'splitter': ['best', 'random'],
                'max_features': ['sqrt', None],
                'max_leaf_nodes': [10, None]
                }
    dtr_grid_search = GridSearchCV(dt, param_grid=params, scoring='neg_mean_squared_error', cv=3, verbose=3, n_jobs=-1)

In [0]:

# Execute GridSearchCV
model = dtr_grid_search.fit(X_train, y_train)


2024/04/03 10:41:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9ac02215bd4142768bfe5daca9ff2a3d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Fitting 3 folds for each of 16 candidates, totalling 48 fits


2024/04/03 10:43:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2024/04/03 10:43:13 INFO mlflow.sklearn.utils: Logging the 5 best runs, 11 runs will be omitted.


In [0]:
# Use the loaded model to make predictions on the test data
y_pred = model.predict(X_test)
# Compare the predicted values with the actual values from the test set
accuracy = mean_squared_error(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5034081778966327
